In [17]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [13]:
data = pd.read_csv("../data/model_ready.csv")

In [21]:
X = data[["Timestamp", "Latitude", "Longitude"]]
y = data[["Magnitude", "Depth"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_scale = StandardScaler()
X_train_scaled = X_scale.fit_transform(X_train)
X_test_scaled = X_scale.transform(X_test)

y_scale = StandardScaler()
y_train_scaled = y_scale.fit_transform(y_train)
y_test_scaled = y_scale.transform(y_test)

(18727, 3) (4682, 3) (18727, 2) (4682, 2)


In [25]:
def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation))
    model.add(Dense(2, activation='softmax'))


    model.compile(optimizer=optimizer, loss=loss, metrics=['mae', 'mse'])

    return model

In [22]:
model = KerasRegressor(model=create_model, verbose=0)

param_grid = {
    'model__neurons': [16, 64],
    'model__activation': ['sigmoid', 'relu'],
    'model__optimizer': ['SGD', 'Adadelta'],
    'model__loss': ['mse', 'mae'],   # better regression losses than square_hinge
    'batch_size': [10],
    'epochs': [10]
}


In [23]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train_scaled)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']


for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

c:\Users\pc\End to End project\Earthquake Prediction\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: -0.221555 using {'batch_size': 10, 'epochs': 10, 'model__activation': 'relu', 'model__loss': 'mse', 'model__neurons': 64, 'model__optimizer': 'SGD'}
-0.245816 (0.011491) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 16, 'model__optimizer': 'SGD'}
-0.274217 (0.020976) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 16, 'model__optimizer': 'Adadelta'}
-0.245018 (0.011163) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 64, 'model__optimizer': 'SGD'}
-0.249913 (0.011789) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 64, 'model__optimizer': 'Adadelta'}
-0.321861 (0.018486) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mae', 'model__neurons': 16, 'model__optimizer': 'SGD'}
-0.271334 (0.011904)

In [26]:
best_model = create_model(neurons=64, activation='relu', optimizer='SGD', loss='mse')
best_model.fit(X_train_scaled, y_train_scaled, epochs=10, batch_size=10, verbose=1)

Epoch 1/10


c:\Users\pc\End to End project\Earthquake Prediction\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2382 - mae: 0.9309 - mse: 1.2382
Epoch 2/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2280 - mae: 0.9273 - mse: 1.2280
Epoch 3/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2249 - mae: 0.9262 - mse: 1.2249
Epoch 4/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2230 - mae: 0.9255 - mse: 1.2230
Epoch 5/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2218 - mae: 0.9247 - mse: 1.2218
Epoch 6/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2210 - mae: 0.9245 - mse: 1.2210
Epoch 7/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2202 - mae: 0.9243 - mse: 1.2202
Epoch 8/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.2195 - mae: 0.9239 - mse: 1.2195
Epoch 9/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.2190 - mae: 0.9239 - mse: 1.2190
Epoch 10/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.2185 - mae: 0.9236 - mse: 1.2185
